In [5]:
!pip install fuzzywuzzy[speedup]

In [9]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from fuzzywuzzy import process, fuzz
import re

# Load data from specific files
health_behavior = pd.read_csv(r"C:\Users\nycmb\OneDrive\Desktop\782\cap\HB.csv")
crime_data = pd.read_excel(r"C:\Users\nycmb\OneDrive\Desktop\782\cap\Crime_data.xlsx", sheet_name='Table 01')

# Clean column names: strip spaces and convert to lowercase
health_behavior.columns = health_behavior.columns.str.strip().str.lower()
crime_data.columns = crime_data.columns.str.strip().str.lower()

# Standardize location names (remove leading/trailing spaces and convert to lowercase)
health_behavior['location'] = health_behavior['location'].str.strip().str.lower()
crime_data['location'] = crime_data['location'].str.strip().str.lower()

# Define a function to get the best match for a location
def get_best_match(location, choices):
    match, score = process.extractOne(location, choices, scorer=fuzz.partial_ratio)
    return match

# Get a list of unique locations from crime data
crime_locations = crime_data['location'].unique()

# Match locations in health_behavior to crime_data using fuzzy matching
health_behavior['matched_location'] = health_behavior['location'].apply(lambda loc: get_best_match(loc, crime_locations))

# Merge datasets on the best match location
data = pd.merge(health_behavior, crime_data, left_on='matched_location', right_on='location', how='left', suffixes=('_hb', '_crime'))

# Handle missing values in crime data
data['crime_count'].fillna(0, inplace=True)

# Calculate Perceived Safety Score
data['perceived_safety_score'] = (data['crime_count'] / data['rate per 100,000 pop']) * 100

# Define feature columns and target
X = data[['age', 'gender', 'matched_location', 'likelihood_percent']]
y = data['behavior']  # Assuming 'behavior' is the target

# Preprocessing for numeric and categorical features
numeric_features = ['likelihood_percent']
categorical_features = ['age', 'gender', 'matched_location']

# Create preprocessing pipelines
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create and configure the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Define an expanded parameter grid for RandomizedSearchCV
param_dist = {
    'classifier__n_estimators': [50, 100, 200, 300, 400, 500],
    'classifier__max_features': ['sqrt', 'log2'],
    'classifier__max_depth': [None, 10, 20, 30, 40, 50],
    'classifier__min_samples_split': [2, 5, 10, 15],
    'classifier__min_samples_leaf': [1, 2, 4, 6],
    'classifier__bootstrap': [True, False]
}

# Set up the RandomizedSearchCV with more iterations
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=20,  # Increase the number of fits
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
random_search.fit(X_train, y_train)

# Get the best model
best_rf_model = random_search.best_estimator_

# Evaluate the model
y_pred = best_rf_model.predict(X_test)
print(classification_report(y_test, y_pred))

# Calculate model accuracy
accuracy = best_rf_model.score(X_test, y_test)
accuracy_percentage = accuracy * 100
print(f"Model Accuracy: {accuracy_percentage:.2f}%")

# Function to parse user query and make predictions
def parse_and_predict(age, gender, location, likelihood_percent):
    # Create a DataFrame from the user input
    user_input = pd.DataFrame({
        'age': [age],
        'gender': [gender],
        'matched_location': [get_best_match(location, crime_locations)],
        'likelihood_percent': [likelihood_percent]
    })
    
    # Make prediction
    prediction = best_rf_model.predict(user_input)
    return prediction[0]

# Function to calculate Perceived Safety Score
def calculate_perceived_safety_score(location):
    # Standardize the location name
    location = location.strip().lower()
    matched_location = get_best_match(location, crime_locations)
    
    # Check if the matched location is in the crime data
    if matched_location in crime_data['location'].values:
        predicted_location_data = crime_data[crime_data['location'] == matched_location]
        crime_count = predicted_location_data['crime_count'].values[0]
        rate_per_100000_pop = predicted_location_data['rate per 100,000 pop'].values[0]
        
        # Ensure rate_per_100000_pop is not zero to avoid division by zero
        if rate_per_100000_pop != 0:
            perceived_safety_score = (crime_count / rate_per_100000_pop) * 100
        else:
            perceived_safety_score = 0
        
        return perceived_safety_score
    else:
        return "Location not found in crime data."

# Function to get safety feedback
def get_safety_feedback(score):
    # Define a threshold for safety
    safety_threshold = 50  # Adjust this threshold based on your criteria
    
    if isinstance(score, str):  # Check if score is an error message
        return score
    elif score >= safety_threshold:
        return "The location is generally safe at night."
    else:
        return "The location might be unsafe at night."

# Function to handle queries
def handle_query(age, gender, location, likelihood_percent):
    predicted_behavior = parse_and_predict(age, gender, location, likelihood_percent)
    print(f'Predicted Behavior: {predicted_behavior}')
    
    # Calculate perceived safety score
    perceived_safety_score = calculate_perceived_safety_score(location)
    print(f'Perceived Safety Score for {location}: {perceived_safety_score}')
    
    # Get safety feedback
    safety_feedback = get_safety_feedback(perceived_safety_score)
    print(f'Safety Feedback: {safety_feedback}')

# Function to parse the question
def parse_question(question):
    # Adjust regex patterns to capture more flexible formats
    patterns = {
        'age': r"age\s*[:=]\s*['\"]([^'\"]+)['\"]",
        'gender': r"gender\s*[:=]\s*['\"]([^'\"]+)['\"]",
        'location': r"location\s*[:=]\s*['\"]([^'\"]+)['\"]",
        'likelihood_percent': r"likelihood_percent\s*[:=]\s*([\d.]+)"
    }

    parsed_data = {}
    
    for key, pattern in patterns.items():
        match = re.search(pattern, question, re.IGNORECASE)
        if match:
            value = match.group(1).strip()
            if key == 'likelihood_percent':
                try:
                    value = float(value)
                except ValueError:
                    value = None
            parsed_data[key] = value

    # Extract values with defaults if not found
    age = parsed_data.get('age', None)
    gender = parsed_data.get('gender', None)
    location = parsed_data.get('location', None)
    likelihood_percent = parsed_data.get('likelihood_percent', None)
    
    # Check if all necessary information is found
    if age and gender and location and likelihood_percent is not None:
        handle_query(age, gender, location, likelihood_percent)
    else:
        print("Could not parse the question. Ensure it is formatted correctly.")

# Example usage with a question
question = "What is the safety prediction for age = '24-35', gender = 'female', location = 'flemington', and likelihood_percent = 18.2?"
parse_question(question)


C:\Users\nycmb\AppData\Local\Temp\ipykernel_20384\3428484023.py:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['crime_count'].fillna(0, inplace=True)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
                 precision    recall  f1-score   support

  Mental Health       1.00      1.00      1.00        34
Physical Health       1.00      1.00      1.00        43
        Smoking       1.00      1.00      1.00        49
         Vaping       1.00      1.00      1.00        34

       accuracy                           1.00       160
      macro avg       1.00      1.00      1.00       160
   weighted avg       1.00      1.00      1.00       160

Model Accuracy: 100.00%
Predicted Behavior: Vaping
Perceived Safety Score for flemington: 166.6167514016859
Safety Feedback: The location is generally safe at night.
